In [316]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import pandas as pd
from pandas import DataFrame
import os

In [317]:
filePath = '../input/jtypeall'
result = os.listdir(filePath)
result = sorted(result)

In [318]:
# pred_name = 'Density'
# pred_name = 'Tn'

In [319]:
# fenzi_name = 'H2'
# fenzi_name = 'OH'
# fenzi_name = 'H2'
# fenzi_name = 'H2'

df = DataFrame()
# H2温度
# pred_name = 'Tn'
# fenzi_name = 'H2'

# H2密度
# pred_name = '(0,0)'
# fenzi_name = 'H2'

# CO温度
# pred_name = 'Tn'
# fenzi_name = 'CO'

# CO密度
# pred_name = 'Density'
# fenzi_name = 'CO'

# OH温度
# pred_name = 'Tn'
# fenzi_name = 'OH'

# OH密度
# pred_name = 'Density'
# fenzi_name = 'OH'

# SiO温度
# pred_name = 'Tn'
# fenzi_name = 'SiO'

# SiO密度
pred_name = 'Density'
fenzi_name = 'SiO'

In [320]:
results = []
results_full = []
# isSample = False
isSample = True

# data_name = 'newNH3'
for j, namesda in enumerate(result):
    # names_use.append(namesda.split('.')[0].split('_')[0]+' '.join(namesda.split('.')[0].split('_')[2:-1]))
    aaaa = namesda.split('.')[0].split('_')
    if len(aaaa) <= 3:
#         if aaaa[1] == data_name:
        results.append(namesda)
    elif len(aaaa) > 3:
#         if aaaa[1] == data_name:
        results_full.append(namesda)
            
results

In [321]:
# results = results[:1]
# results

In [322]:
df = DataFrame()

In [323]:
import time
from tqdm import tqdm

for i, fold_name in enumerate(tqdm(results)):
#   if i == 3:
#     continue
#   else:
#     if not os.path.exists('../input/jtypeall/'+fold_name+'/H2_lev.out'):
    if not os.path.exists(f'../input/jtypeall/'+fold_name+f'/{fenzi_name}_lev.out'):
      continue
    else:
      CO_cdens = pd.read_csv(f'../input/jtypeall/'+fold_name+f'/{fenzi_name}_lev.out', header=None, engine='python')
      df_new = DataFrame()

      fi_result, line1, line2, line3, line4, line5, names, finalo = [], [], [], [], [], [], [], []

      for i, item in enumerate(CO_cdens[0]):
          if i == 0:
              remove_space = item.split(' ')
              remove_none = [x for x in remove_space if x !='']
              names.append(remove_none[:5])
          else:
            remove_space = item.split(' ')
            remove_none = [float(x) for x in remove_space if x !='']
            finalo.append(remove_none[:5])

          # print(remove_none[:3])

          if i == 0:
              names.append(remove_none[:5])
          else:
              fi_result.append(remove_none[:5])

      for j in fi_result:
          line1.append(j[0])
          line2.append(j[1])
          line3.append(j[2])
          line4.append(j[3])
          line5.append(j[4])

      line1, line2, line3, line4, line5 = pd.Series(line1), pd.Series(line2), pd.Series(line3), pd.Series(line4), pd.Series(line5)

      df_new.insert(0, 'distance', line1)
      df_new.insert(1, 'timeN', line2)
      if pred_name == 'Tn':
          df_new.insert(2, f'{pred_name}', line3)
      elif pred_name == '(0,0)':
          df_new.insert(2, f'{pred_name}', line4)
      else:
          df_new.insert(2, f'{pred_name}', line5)
      # df.insert(3, '(0,0)', line4)
    
      fig = plt.figure(figsize = (10,4))
      ax1 = fig.add_subplot(1,2,1)
      ax1.scatter(df_new['distance'], df_new[f'{pred_name}'])
      plt.xlabel('distance')
      plt.ylabel(f'{pred_name}')
      plt.grid()
      # 正线性相关

      ax2 = fig.add_subplot(1,2,2)
      ax2.scatter(df_new['timeN'], df_new[f'{pred_name}'])
      plt.xlabel('timeN')
      plt.ylabel(f'{pred_name}')
      plt.grid()
      plt.savefig(f'./sandiantu_{pred_name}.png', dpi=300)
        
      df = df.append(df_new)

In [324]:
df.reset_index(drop=True, inplace=True)

In [325]:
df = df.dropna(axis=0, how='any')

In [326]:
df

In [327]:
features_3 = ['distance', 'timeN', f'{pred_name}']
# features_3 = ['distance', 'timeN', 'Density']
features = df[features_3]


features.plot(subplots=True)
fig = plt.gcf()
fig.savefig('./j-type-data.png', dpi=300)

In [328]:
test_split=round(len(df)*0.2)

In [329]:
test_split = test_split - 1000

In [330]:
df_for_training=df[:-test_split]
df_for_testing=df[-test_split:-1000]
df_30_days_future=df[-1000:]

In [331]:
print(df_for_training.shape)
print(df_for_testing.shape)
print(df_30_days_future.shape)

In [332]:
scaler = MinMaxScaler(feature_range=(0,1))
df_for_training_scaled = scaler.fit_transform(df_for_training)

In [333]:
scaler

In [334]:
df_for_testing_scaled=scaler.transform(df_for_testing)

In [335]:
df_for_training_scaled

In [336]:
back_num = 1000

In [337]:
def createXY(dataset,n_past):
    dataX = []
    dataY = []
    for i in range(n_past, len(dataset)):
            dataX.append(dataset[i - n_past:i, 0:dataset.shape[1]])
            dataY.append(dataset[i,2])
    return np.array(dataX),np.array(dataY)        

In [338]:
# df_for_training_scaled = finalo[:-782, :]
trainX,trainY=createXY(df_for_training_scaled,back_num)

In [339]:
trainX.shape

In [340]:
# df_for_testing_scaled = finalo[-782:, :]
testX,testY=createXY(df_for_testing_scaled,back_num)

In [341]:
trainX[0].shape

In [342]:
print("trainX Shape-- ",trainX.shape)
print("trainY Shape-- ",trainY.shape)

In [343]:
print("testX Shape-- ",testX.shape)
print("testY Shape-- ",testY.shape)

In [344]:
print("trainX[0]-- \n",trainX[0])
print("\ntrainY[0]-- ",trainY[0])

In [345]:
trainY[0]

In [346]:
trainY.shape

In [347]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV

In [348]:
def build_model(optimizer):
    grid_model = Sequential()
    grid_model.add(LSTM(100,return_sequences=True,input_shape=(back_num,3)))
    grid_model.add(LSTM(100))
    grid_model.add(Dropout(0.1))
    grid_model.add(Dropout(0.2))
    grid_model.add(Dropout(0.3))
    grid_model.add(Dropout(0.4))
    grid_model.add(Dropout(0.5))
    # grid_model.add(Dropout(0.1))
    grid_model.add(Dense(1))

    grid_model.compile(loss = 'mae',optimizer = optimizer)
#     grid_model.summary()
    return grid_model

grid_model = KerasRegressor(build_fn=build_model,verbose=1,validation_data=(testX,testY))
parameters = {'batch_size' : [256],
# parameters = {'batch_size' : [20],
              'epochs' : [10],
              'optimizer' : ['adam'] }

grid_search  = GridSearchCV(estimator = grid_model,
                            param_grid = parameters,
                            cv = 3)

In [349]:
grid_search = grid_search.fit(trainX,trainY)

In [350]:
grid_search.best_params_

In [351]:
my_model=grid_search.best_estimator_.model
prediction=my_model.predict(testX)

In [352]:
print("\nPrediction Shape-",prediction.shape)
prediction_copies_array = np.repeat(prediction, 3, axis=-1)

In [353]:
pred=scaler.inverse_transform(np.reshape(prediction_copies_array,(len(prediction),3)))[:,2]

In [354]:
original_copies_array = np.repeat(testY, 3, axis=-1)

original_copies_array.shape

original=scaler.inverse_transform(np.reshape(original_copies_array,(len(testY),3)))[:,2]

In [355]:
print("Pred Values-- " ,pred.shape)
print("\nOriginal Values-- ",original.shape)

In [356]:
import matplotlib.pyplot as plt

In [357]:
plt.plot(original, color = 'red', label = 'Real')
plt.plot(pred, color = 'blue', label = 'Predicted')
plt.title('Prediction Compare')
plt.xlabel('Sequence Order')
if pred_name == 'Tn':
    plt.ylabel(f'Temperature {pred_name}')
elif pred_name == '(0,0)':
    plt.ylabel(f'Density {pred_name}')
else:
    plt.ylabel(f'{pred_name}')
# plt.yticks(np.arange(-3, -2, 0.1)) 
plt.legend()
plt.savefig(f'./J_type_{fenzi_name}_Predition_{pred_name}', dpi=300)
plt.show()

# END!!!!

In [358]:
df_30_days_past=df_for_testing.iloc[-back_num:,:]
df_30_days_past.tail()

In [359]:
df_30_days_future=df[-back_num:]

In [360]:
df_30_days_future

In [361]:
# df_30_days_future=df_30_days_future[["distance","timeN","(0,0)"]]
df_30_days_future[f'{pred_name}']=0

In [362]:
df_30_days_future

In [363]:
old_scaled_array=scaler.transform(df_30_days_past)
new_scaled_array=scaler.transform(df_30_days_future)
new_scaled_df=pd.DataFrame(new_scaled_array)
new_scaled_df.iloc[:,2]=np.nan

In [364]:
full_df=pd.concat([pd.DataFrame(old_scaled_array),new_scaled_df]).reset_index().drop(["index"], axis=1)

In [365]:
full_df

In [366]:
full_df_scaled_array=full_df.values

In [367]:
full_df_scaled_array

In [368]:
full_df_scaled_array.shape

In [369]:
all_data=[]
time_step=back_num
for i in range(time_step,len(full_df_scaled_array)):
    data_x=[]
    data_x.append(full_df_scaled_array[i-time_step:i,0:full_df_scaled_array.shape[1]])
    data_x=np.array(data_x)
    prediction=my_model.predict(data_x)
    all_data.append(prediction)
    full_df.iloc[i,2]=prediction

In [370]:
all_data

In [371]:
new_array=np.array(all_data)
new_array=new_array.reshape(-1,1)
prediction_copies_array = np.repeat(new_array, 3, axis=-1)
y_pred_future_30_days = scaler.inverse_transform(np.reshape(prediction_copies_array,(len(new_array),3)))[:,2]

In [372]:
original_resul=np.array(df[-back_num:][f'{pred_name}'])

In [373]:
# # plt.plot(original_resul, color = 'red', label = 'Real')
# plt.plot(y_pred_future_30_days, color = 'blue', label = 'Predicted 1000 steps')
# plt.title('Prediction Compare')
# plt.xlabel('Sequence Order')
# if pred_name == 'Tn':
#     plt.ylabel(f'Temperature {pred_name}')
# elif pred_name == '(0,0)':
#     plt.ylabel(f'Density {pred_name}')
# else:
#     plt.ylabel(f'{pred_name}')
# plt.legend()
# plt.savefig(f'./J_type_{fenzi_name}_Predition_result_{pred_name}', dpi=300)
# plt.show()

In [374]:
sds = [np.nan]*len(pred)
aagsa = np.concatenate((sds, y_pred_future_30_days), axis=0)

plt.plot(original, color = 'red', label = 'Real')
plt.plot(pred, color = 'blue', label = 'LSTM Fitting')
plt.plot(aagsa, color = 'green', label = 'Predicted')
plt.title('Prediction Compare')
plt.xlabel('Sequence Order')
if pred_name == 'Tn':
    plt.ylabel(f'Temperature {pred_name}')
elif pred_name == '(0,0)':
    plt.ylabel(f'Density {pred_name}')
else:
    plt.ylabel(f'{pred_name}')
# plt.yticks(np.arange(-3, -2, 0.1)) 
plt.legend()
plt.savefig(f'./J_type_{fenzi_name}_Predition_{pred_name}', dpi=300)
plt.show()

In [375]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model

In [376]:
my_model.save(f'/content/drive/MyDrive/NLU/Multivariate-time-series-forecasting-using-LSTM-main/Jtype_{pred_name}_Model_future_value.h5')
print('Model Saved!')

In [377]:
scaler

In [378]:
import pickle
scalerfile = f'/content/drive/MyDrive/NLU/Multivariate-time-series-forecasting-using-LSTM-main/Jtype_{pred_name}_scaler_model_future_value.pkl'
pickle.dump(scaler, open(scalerfile, 'wb'))